In [6]:
import numpy as np
import pandas as pd


In [7]:
placement_data = pd.read_csv(r"C:\Users\HAI\Downloads\placedata v2.0 synthetic.csv", index_col= False)
placement_data.head()

,StudentID,CGPA,Internships,Projects,Workshops/Certifications,AptitudeTestScore,SoftSkillsRating,ExtracurricularActivities,PlacementTraining,SSC_Marks,HSC_Marks,PlacementStatus
0,1,7.5,1,1,1,65,4.4,No,No,61,79,NotPlaced
1,2,8.9,0,3,2,90,4.0,Yes,Yes,78,82,Placed
2,3,7.3,1,2,2,82,4.8,Yes,No,79,80,NotPlaced
3,4,7.5,1,1,2,85,4.4,Yes,Yes,81,80,Placed
4,5,8.3,1,2,2,86,4.5,Yes,Yes,74,88,Placed


In [8]:
# DROPING THE COLUMN THAT NOT REQUIRED FOR TRAINING MODEL
placement_data = placement_data.drop('StudentID',axis = 1)
placement_data.head()

,CGPA,Internships,Projects,Workshops/Certifications,AptitudeTestScore,SoftSkillsRating,ExtracurricularActivities,PlacementTraining,SSC_Marks,HSC_Marks,PlacementStatus
0,7.5,1,1,1,65,4.4,No,No,61,79,NotPlaced
1,8.9,0,3,2,90,4.0,Yes,Yes,78,82,Placed
2,7.3,1,2,2,82,4.8,Yes,No,79,80,NotPlaced
3,7.5,1,1,2,85,4.4,Yes,Yes,81,80,Placed
4,8.3,1,2,2,86,4.5,Yes,Yes,74,88,Placed


In [9]:
# defining feature and target

y = placement_data.PlacementStatus
# X = train_set
X = placement_data.drop(['PlacementStatus'],axis = 1)

In [10]:
# preprocessing categorical data into 0's and 1's

X['ExtracurricularActivities'] = X['ExtracurricularActivities'].str.strip().replace({'Yes': 1, 'No': 0})
X['PlacementTraining'] = X['PlacementTraining'].str.strip().replace({'Yes': 1, 'No': 0})
y = y.str.strip().replace({'Placed': 1, 'NotPlaced': 0})

In [11]:
# splitting the data into training and validation sets 

from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size = 0.2,random_state = 1)

In [12]:
# making pipelines for preprocessing 
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# model pipeline
from xgboost import XGBClassifier

p1 = Pipeline(steps = [('prep',SimpleImputer(strategy= 'mean')),('xgbc',XGBClassifier(n_estimators = 1000,learning_rate = 0.005,objective="binary:logistic", eval_metric="logloss",random_state = 1))])
p1.fit(X_train,y_train)

from sklearn.model_selection import cross_val_score

score = cross_val_score(p1 ,X_train,y_train,cv = 4,scoring = 'accuracy',n_jobs = 4)
print(score.mean())

C:\Users\HAI\anaconda3\envs\tf\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.7913749999999999


In [14]:
preds = p1.predict(X_val)

In [15]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_val,preds)
acc


0.8115

In [16]:
confusion_matrix = pd.crosstab(y_val,preds)

In [17]:
confusion_matrix

col_0,0,1
PlacementStatus,,
0,1008,184
1,193,615


In [65]:
#save as pickle
import pickle
pickle.dump(p1, open("placement_model_v1.pkl", "wb"))